# Ibis MapD Backend - Test Numeric

In [1]:
from ibis import literal as L
from ibis.tests.backends import MapD

import ibis
import math
import numpy as np

ibis.options.interactive = False

In [2]:
mapd_cli = ibis.mapd.connect(
    host='localhost', user='mapd', password='HyperInteractive',
    port=9091, database='ibis_testing'
)
backend = MapD(None)

In [3]:
def param(ibis_fn, pandas_fn, id, skip=False):
    if skip:
        return None, None, None
    return ibis_fn, pandas_fn, id

In [4]:
alltypes = mapd_cli.database().functional_alltypes
df = alltypes.execute()

In [5]:
tests_1_operand = (
    param(lambda t: t.float_col,
          lambda t: t.float_col,
          id='float-column-isinf'),
    param(lambda t: t.double_col,
          lambda t: t.double_col,
          id='double-column'),
    param(lambda t: ibis.literal(1.3),
          lambda t: 1.3,
          id='float-literal'),  # strange fail on Postgres
    param(lambda t: ibis.literal(np.nan),
          lambda t: np.nan,
          id='nan-literal'),
    param(lambda t: ibis.literal(np.inf),
          lambda t: np.inf,
          id='inf-literal'),
    param(lambda t: ibis.literal(-np.inf),
          lambda t: -np.inf,
          id='-inf-literal')
)

tests_1_expr = (
    param(lambda o: o.isnan(), lambda o: np.isnan(o), id='isnan'),
    param(lambda o: o.isinf(), lambda o: np.isinf(o), id='isinf')
)

In [6]:
for operand_fn, expected_operand_fn, id_test_operand in tests_1_operand:
    for expr_fn, expected_expr_fn, id_test_expr in tests_1_expr:
        if operand_fn is None or expr_fn is None:
            continue
        print('{}-{} ... '.format(id_test_operand, id_test_expr), end='')
        
        try:
            expr = expr_fn(operand_fn(alltypes))
            expected = expected_expr_fn(expected_operand_fn(df))

            result = mapd_cli.execute(expr)

            if isinstance(expected, pd.Series):
                expected = backend.default_series_rename(expected)
                backend.assert_series_equal(result, expected)
            else:
                assert result == expected
            print('OK!')
        except ibis.common.UnsupportedOperationError as e:
            print('UnsupportedOperationError')

float-column-isinf-isnan ... UnsupportedOperationError
float-column-isinf-isinf ... UnsupportedOperationError
double-column-isnan ... UnsupportedOperationError
double-column-isinf ... UnsupportedOperationError
float-literal-isnan ... UnsupportedOperationError
float-literal-isinf ... UnsupportedOperationError
nan-literal-isnan ... UnsupportedOperationError
nan-literal-isinf ... UnsupportedOperationError
inf-literal-isnan ... UnsupportedOperationError
inf-literal-isinf ... UnsupportedOperationError
-inf-literal-isnan ... UnsupportedOperationError
-inf-literal-isinf ... UnsupportedOperationError


In [7]:
test_2 = (
    param(L(-5).abs(), 5, id='L(-5).abs()'),
    param(L(5).abs(), 5, id='L(5).abs()'),
    param(ibis.least(L(10), L(1)), 1, id='ibis.least(L(10), L(1))'),
    param(ibis.greatest(L(10), L(1)), 10, id='ibis.greatest(L(10), L(1))'),
    param(L(5.5).round(), 6.0, id='L(5.5).round()'),
    param(L(5.556).round(2), 5.56, id='L(5.556).round(2)'),
    param(L(5.556).ceil(), 6.0, id='L(5.556).ceil()'),
    param(L(5.556).floor(), 5.0, id='L(5.556).floor()'),
    param(L(5.556).exp(), math.exp(5.556), id='L(5.556).exp()'),
    param(L(5.556).sign(), 1, id='L(5.556).sign()'),
    param(L(-5.556).sign(), -1, id='L(-5.556).sign()'),
    param(L(0).sign(), 0, id='L(0).sign()'),
    param(L(5.556).sqrt(), math.sqrt(5.556), id='L(5.556).sqrt()'),
    param(L(5.556).log(2), math.log(5.556, 2), id='L(5.556).log(2)'),
    param(L(5.556).ln(), math.log(5.556), id='L(5.556).ln()'),
    param(L(5.556).log2(), math.log(5.556, 2), id='L(5.556).log2()'),
    param(L(5.556).log10(), math.log10(5.556), id='L(5.556).log10()'),
    param(L(11) % 3, 11 % 3, id='L(11) % 3')
)

In [8]:
for expr, expected, id_test in test_2:
    if expr is None:
        continue
        
    print('{} ... '.format(id_test), end='')

    try:
        result = mapd_cli.execute(expr)

        if isinstance(result, decimal.Decimal):
            # in case of Impala the result is decimal
            # >>> decimal.Decimal('5.56') == 5.56
            # False
            assert result == decimal.Decimal(str(expected))
        else:
            assert result == expected
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        print(e)

L(-5).abs() ... Exception: Non-empty LogicalValues not supported yet: SELECT abs(-5) AS tmp
L(5).abs() ... Exception: Non-empty LogicalValues not supported yet: SELECT abs(5) AS tmp
ibis.least(L(10), L(1)) ... UnsupportedOperationError
ibis.greatest(L(10), L(1)) ... UnsupportedOperationError
L(5.5).round() ... Exception: Non-empty LogicalValues not supported yet: SELECT round(5.5) AS tmp
L(5.556).round(2) ... Exception: Non-empty LogicalValues not supported yet: SELECT round(5.556, 2) AS tmp
L(5.556).ceil() ... Exception: Non-empty LogicalValues not supported yet: SELECT ceil(5.556) AS tmp
L(5.556).floor() ... Exception: Non-empty LogicalValues not supported yet: SELECT floor(5.556) AS tmp
L(5.556).exp() ... Exception: Non-empty LogicalValues not supported yet: SELECT exp(5.556) AS tmp
L(5.556).sign() ... Exception: Non-empty LogicalValues not supported yet: SELECT sign(5.556) AS tmp
L(-5.556).sign() ... Exception: Non-empty LogicalValues not supported yet: SELECT sign(-5.556) AS tmp
L